In [101]:
import requests
import numpy as np
import pandas as pd
import json
import random
import codecs
import unicodedata
from bs4 import BeautifulSoup
from translate import Translator

In [87]:
def getMovies(): 
  url = 'https://www.tamilpaa.com/tamil-movies-list'
  r = requests.get(url)
  content = r.content
  soup = BeautifulSoup(content)
  detail_content = soup.find('table', attrs = {'class': 'standard mb-50px'})
  tablecontent = detail_content.find_all('tr')
  movies = []
  for tr in tablecontent[1:]:
    song_details = {}
    a_tags = tr.find_all('a')
    td_tags = tr.find_all('td')
    
    song_details["movie_url"] = a_tags[0].get('href')
    song_details["year"] = td_tags[1].get_text()
    song_details["song_music"] = td_tags[2].get_text()
    song_details["actors"] = td_tags[3].get_text()
    movies.append(song_details)
  return movies

In [88]:
def getAllSongByMovie(movie):
  songs_data = []
  r = requests.get(movie["movie_url"])
  content = r.content
  soup = BeautifulSoup(content)
  detail_content = soup.find('table', attrs = {'class': 'standard'})
  td_tags = detail_content.find_all('td')
  movie["movie_name"]= td_tags[2].get_text()
  song_div = soup.find('div', attrs = {'class': 'tab-content clearfix', 'id': 'tab_1'})
  song_li_tags =song_div.find_all('li')
  
  for song_li in song_li_tags:
    song_details ={}
    song_details["song_name"]= song_li.find('a').get_text()
    song_details["song_music"] = movie["song_music"]
    song_details["song_url"]= song_li.find('a').get('href')
    songs_data.append(song_details)
  movie["movie_song"]=songs_data
  return movie

In [89]:
def getSingersAndLyricsBySong(song):
  r = requests.get(song["song_url"])
  content = r.content
  soup = BeautifulSoup(content)
  detail_content = soup.find('table', attrs = {'class': 'standard mb-10px'})
  tablecontent =detail_content.find_all('td')
  song["song_singers"]= tablecontent[4].get_text()
  song_content = soup.find('div', attrs = {'class': 'info-box white-bg'})
  song["song_lyrics"]= (song_content.get_text()).strip()
  del song["song_url"]
  return song

In [ ]:
movies_List=getMovies()
a=0
for movie in movies_List:
  movie1=getAllSongByMovie(movie)
  songs=movie1["movie_song"]
  b=0
  for song in songs:
    songs[b]=getSingersAndLyricsBySong(song)
    b+=1
  movies_List[a]["movie_song"]=songs
  del movies_List[a]["movie_url"]
  a+=1


In [102]:
f = codecs.open('tamil_songs_scraped_lyrics.txt', 'w', encoding='utf-8')
for line in movies_List:
  try:
      song_json = str(line).replace("\'", "\"")
      f.write(song_json)
      f.write('\n')
  except Exception as e:
      print(e)